In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from fastml.core import *
from fastml.data.datasets import *
from fastml.model import loss as loss
from fastml.model import metrics as metrics
from fastml.model import optimizers
from fastml.model.callbacks import *
from torch import nn

In [3]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = loss.mse
        
    def __call__(self, x, targ):
        for layer in self.layers: x = layer(x)
        return self.loss(x.squeeze(), targ)

In [4]:
train_ds,valid_ds = Datasets.MNIST()

In [5]:
train_ds.x.shape

torch.Size([50000, 784])

In [6]:
model = Model(784, 50, 1)

## Training

In [7]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.l1 = nn.Linear(n_in, nh)
        self.l2 = nn.Linear(nh, n_out)

    def __call__(self, x): return self.l2(F.relu(self.l1(x)))

In [8]:
for name,l in model.named_children(): print(f"{name}: {l}")

In [9]:
model

Model()

In [10]:
bs=64 # batch size

model = Model(784, 50, 10)
xb, yb = train_ds[0:bs] # a mini-batch from x
preds = model(xb)      # predictions

preds[0], preds.shape

(tensor([-0.2254,  0.4871, -0.4744,  0.0137, -0.2635,  0.3699,  0.4965,  0.0540,
          0.4427, -0.3002], grad_fn=<SelectBackward>), torch.Size([64, 10]))

In [11]:
accuracy = metrics.accuracy(preds, yb[0:bs])

## basic train

In [12]:
x_train, y_train = train_ds.x, train_ds.y
x_valid, y_valid = valid_ds.x, valid_ds.y

In [13]:
lr = 0.5
epochs = 2
batch_size=64
loss_func = loss.cross_entropy()
model = Model(784, 50, 10)

def fit():
    for epoch in range(epochs):
        for batch in range(x_train.shape[0]//batch_size):
            
            b_from = batch*batch_size
            b_to = b_from + batch_size
            x_batch, y_batch = train_ds[b_from:b_to]
            
            loss = loss_func(model(x_batch), y_batch)
            
            loss.backward()
            with torch.no_grad():
                for p in model.parameters(): p -= lr * p.grad
                model.zero_grad()
            
        accuracy = metrics.accuracy(model(x_train), y_train)
        print("epoch %s accuracy is %f3" % (epoch, accuracy))

In [14]:
def assert_train_accuracy():
    assert metrics.accuracy(model(x_train), y_train) > 80

In [15]:
fit()
assert_train_accuracy()

epoch 0 accuracy is 90.4500013
epoch 1 accuracy is 92.9000023


In [16]:
class SequentialModel(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = nn.ModuleList(layers)
        
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [17]:
layers = [nn.Linear(784,50), nn.ReLU(), nn.Linear(50,10)]
model = SequentialModel(layers)

fit()
assert_train_accuracy()

epoch 0 accuracy is 88.9259993
epoch 1 accuracy is 89.8760023


In [18]:
model = nn.Sequential(nn.Linear(784,50), nn.ReLU(), nn.Linear(50,10))

fit()
assert_train_accuracy()

epoch 0 accuracy is 91.6260003
epoch 1 accuracy is 92.7339973


In [19]:
model

Sequential(
  (0): Linear(in_features=784, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=10, bias=True)
)

In [20]:
def create_model():
    return nn.Sequential(nn.Linear(784,50), nn.ReLU(), nn.Linear(50,10))

In [21]:
model = create_model()

fit()
assert_train_accuracy()

epoch 0 accuracy is 88.6420013
epoch 1 accuracy is 91.5080013


## with optimizer

In [22]:
model = create_model()
optimizer = optimizers.SGD(model)

In [23]:
def fit():
    for epoch in range(epochs):
        for batch in range(x_train.shape[0]//batch_size):
            
            b_from = batch*batch_size
            b_to = b_from + batch_size
            x_batch, y_batch = train_ds[b_from:b_to]
            
            loss = loss_func(model(x_batch), y_batch)
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        accuracy = metrics.accuracy(model(x_train), y_train)
        print("epoch %s accuracy is %f3" % (epoch, accuracy))

In [24]:
fit()
assert_train_accuracy()

epoch 0 accuracy is 91.1840023
epoch 1 accuracy is 91.9460003


## with data loader

In [25]:
train_dl,_ = get_data_loaders(train_ds, valid_ds, batch_size)

In [26]:
model = create_model()
optimizer = optimizers.SGD(model)

def fit():
    for epoch in range(epochs):
        for x_batch, y_batch in train_dl:
            
            loss = loss_func(model(x_batch), y_batch)
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        accuracy = metrics.accuracy(model(x_train), y_train)
        print("epoch %s accuracy is %f3" % (epoch, accuracy))

In [27]:
fit()
assert_train_accuracy()

epoch 0 accuracy is 92.4279993
epoch 1 accuracy is 89.7840023


## with validation

In [28]:
def assert_valid_accuracy():
    assert metrics.accuracy(model(x_valid), y_valid) > 80

In [29]:
def fit(epochs, model, train_dl, valid_dl, optimizer, loss_func):
    for epoch in range(epochs):
        
        # set model to train_mode
        model.train()
        
        for x_batch, y_batch in train_dl:
        
            loss = loss_func(model(x_batch), y_batch)
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        # set model to eval_mode
        model.eval()
        with torch.no_grad():
            total_accuracy, total_loss = 0., 0.
            for x_valid_batch, y_valid_batch in valid_dl:
                valid_preds = model(x_valid_batch)
                total_loss+=loss_func(valid_preds, y_valid_batch)
                total_accuracy+=metrics.accuracy(valid_preds, y_valid_batch)
                
        n = len(valid_dl)        
        print("epoch %s validation accuracy: %f3 loss: %f3" % (epoch, total_accuracy/n, total_loss.item()/n))

In [30]:
train_dl, valid_dl = get_data_loaders(train_ds, valid_ds, batch_size)
model = create_model()
optimizer = optimizers.SGD(model)
loss_func = loss.cross_entropy()

fit(5, model, train_dl, valid_dl, optimizer, loss_func)
assert_valid_accuracy()

epoch 0 validation accuracy: 81.8910263 loss: 0.6780233
epoch 1 validation accuracy: 92.4879813 loss: 0.3591723
epoch 2 validation accuracy: 95.9635423 loss: 0.1665853
epoch 3 validation accuracy: 96.2640223 loss: 0.1649593
epoch 4 validation accuracy: 96.5344553 loss: 0.1715903


## Learner

In [31]:
#export
class Learner():
    def __init__(self, model, optimizer, loss_func, data):
        self.model,self.optimizer,self.loss_func,self.data = model,optimizer,loss_func,data

In [32]:
def get_simple_lin_model(data, lr=0.3, nh=50):
    out_classes = data.train_ds.y.max().item()+1
    in_ = data.train_ds.x.shape[1]
    model = nn.Sequential(nn.Linear(in_,nh), nn.ReLU(), nn.Linear(nh,out_classes))
    return model, optimizers.SGD(model, lr=lr)

In [33]:
data = data_bunch(train_ds,valid_ds, 64)
learner = Learner(*get_simple_lin_model(data), loss.cross_entropy(), data)

In [34]:
def fit(epochs, learner):
    for epoch in range(epochs):
        
        # set model to train_mode
        learner.model.train()
        
        for x_batch, y_batch in learner.data.train_dl:
        
            loss = learner.loss_func(learner.model(x_batch), y_batch)
            
            loss.backward()
            learner.optimizer.step()
            learner.optimizer.zero_grad()
            
        # set model to eval_mode
        learner.model.eval()
        with torch.no_grad():
            total_accuracy, total_loss = 0., 0.
            for x_valid_batch, y_valid_batch in learner.data.valid_dl:
                valid_preds = learner.model(x_valid_batch)
                total_loss+=learner.loss_func(valid_preds, y_valid_batch)
                total_accuracy+=metrics.accuracy(valid_preds, y_valid_batch)
                
        n = len(valid_dl)        
        print("epoch %s validation accuracy: %f3 loss: %f3" % (epoch, total_accuracy/n, total_loss.item()/n))

In [35]:
fit(5, learner)
assert_valid_accuracy()

epoch 0 validation accuracy: 96.7748403 loss: 0.1289533
epoch 1 validation accuracy: 96.5645033 loss: 0.1389913
epoch 2 validation accuracy: 97.1053693 loss: 0.1230163
epoch 3 validation accuracy: 88.0709133 loss: 0.6151113
epoch 4 validation accuracy: 97.4859783 loss: 0.1289313


## Runner

In [36]:
#export
class CancelTrainException(Exception): pass
class CancelEpochException(Exception): pass
class CancelBatchException(Exception): pass

In [37]:
#export
class Runner():
    def __init__(self, cbs=None, cb_funcs=None):
        cbs = listify(cbs)
        for cbf in listify(cb_funcs):
            cb = cbf()
            setattr(self, cb.name, cb)
            cbs.append(cb)
        self.stop,self.cbs = False,[TrainEvalCallback()]+cbs

    @property
    def optimizer(self): return self.learn.optimizer
    @property
    def model(self):     return self.learn.model
    @property
    def loss_func(self): return self.learn.loss_func
    @property
    def data(self):      return self.learn.data
    
    def fit(self, epochs, learn):
        self.epochs,self.learn,self.loss = epochs,learn,tensor(0.)

        try:
            for cb in self.cbs: cb.set_runner(self)
            self('begin_fit')
            for epoch in range(epochs):
                self.epoch = epoch
                if not self('begin_epoch'): self.train_with_batches(self.data.train_dl)
                with torch.no_grad(): 
                    if not self('begin_validate'): self.train_with_batches(self.data.valid_dl)
                self('after_epoch')
        except CancelTrainException: self('after_cancel_train')
        finally:
            self('after_fit')
            self.learn = None
            
    def train_with_batches(self, dl):
        self.iters = len(dl)
        try:
            for xb,yb in dl: self.train_with_one_batch(xb, yb)
        except CancelEpochException: self('after_cancel_epoch')

    def train_with_one_batch(self, xb, yb):
        try:
            self.xb,self.yb = xb,yb
            self('begin_batch')
            self.pred = self.model(self.xb)
            self('after_pred')
            self.loss = self.loss_func(self.pred, self.yb)
            self('after_loss')
            
            if not self.in_train: return
            
            self.loss.backward()
            self('after_backward')
            self.optimizer.step()
            self('after_step')
            self.optimizer.zero_grad()
        except CancelBatchException: self('after_cancel_batch')
        finally: self('after_batch')

    def __call__(self, cb_name):
        res = False
        for cb in sorted(self.cbs, key=lambda x: x._order): res = cb(cb_name) or res
        return res

In [38]:
data = data_bunch(train_ds,valid_ds, 64)
learner = Learner(*get_simple_lin_model(data), loss.cross_entropy(), data)

stats = AvgStatsCallback([metrics.accuracy])
run = Runner(cbs=stats)
run.fit(5, learner)

train: [0.27009966796875, 91.696]
valid: [0.279593408203125, 91.34]
train: [0.134194267578125, 96.002]
valid: [0.53961279296875, 90.08]
train: [0.104303935546875, 96.882]
valid: [0.11928544921875, 96.74]
train: [0.08923349609375, 97.234]
valid: [0.13650467529296875, 96.36]
train: [0.0741594140625, 97.696]
valid: [0.2293269287109375, 94.62]


In [42]:
#export
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x): return self.func(x)

## Export

In [44]:
!python notebook2script.py model_dev.ipynb fastml/model/model.py

Converted model_dev.ipynb to fastml/model/model.py
